In [75]:
import tifffile
import numpy as np


# Method 1: Using tifffile
original_image = tifffile.imread(r'G:\Alexis_data\Project\FISH\input\Heart2_round3_DAPI_resliced-q1-z1_cp_masks.tif')

# Print image shape and data type
print(f"Image shape: {original_image.shape}")
print(f"Data type: {original_image.dtype}")

# Display basic statistics
print(f"Min value: {np.min(original_image)}")
print(f"Max value: {np.max(original_image)}")
print(f"Mean value: {np.mean(original_image)}")

Image shape: (390, 1164, 1024)
Data type: uint16
Min value: 0
Max value: 15334
Mean value: 367.99247905110377


In [118]:
def segment_3d_with_coordinates(data, partition_d=2, partition_h=2, partition_w=2):
    """
    Segments 3D data and returns both segments and their coordinates
    
    Returns:
        main_segments: list of regular-sized segments
        main_coordinates: list of (d_start, d_end, h_start, h_end, w_start, w_end)
        leftover_segments: list of leftover segments
        leftover_coordinates: list of coordinates for leftover segments
    """
    depth, height, width = data.shape
    

    # # If height_size and width_size are tuples, get their first elements
    # height_size = height if isinstance(height_size, tuple) else height
    # width_size = width if isinstance(width_size, tuple) else width
    # depth_size = depth if isinstance(depth_size, tuple) else depth

    # Now perform the integer division
    n_depth = partition_d = 2
    n_height = partition_h = 2
    n_width = partition_w = 2

    depth_size = depth // partition_d
    height_size = height // partition_h
    width_size = width // partition_w

    # n_height = height // 
    # n_width = width // width_size

    main_segments = []
    main_coordinates = []
    leftover_segments = []
    leftover_coordinates = []
    
    # Extract main segments with coordinates
    for d in range(n_depth):
        for h in range(n_height):
            for w in range(n_width):
                d_start = d * depth_size
                d_end = (d + 1) * depth_size
                h_start = h * height_size
                h_end = (h + 1) * height_size
                w_start = w * width_size
                w_end = (w + 1) * width_size
                
                segment = data[d_start:d_end, h_start:h_end, w_start:w_end]
                main_segments.append(segment)
                main_coordinates.append((d_start, d_end, h_start, h_end, w_start, w_end))
    
    # Handle leftover in depth
    if depth % depth_size != 0:
        d_start = n_depth * depth_size
        d_end = depth
        h_end = height_size * n_height
        w_end = width_size * n_width
        
        leftover_d = data[d_start:d_end, :h_end, :w_end]
        if leftover_d.size > 0:
            leftover_segments.append(leftover_d)
            leftover_coordinates.append((d_start, d_end, 0, h_end, 0, w_end))

    
    # Handle leftover in height
    if height % height_size != 0:
        d_end = depth_size * n_depth
        h_start = height_size * n_height
        h_end = height
        w_end = width_size * n_width
        
        leftover_h = data[:d_end, h_start:h_end, :w_end]
        if leftover_h.size > 0:
            leftover_segments.append(leftover_h)
            leftover_coordinates.append((0, d_end, h_start, h_end, 0, w_end))
    
    # Handle leftover in width
    if width % width_size != 0:
        d_end = depth_size * n_depth
        h_end = height_size * n_height
        w_start = width_size * n_width
        w_end = width
        
        leftover_w = data[:d_end, :h_end, w_start:w_end]
        if leftover_w.size > 0:
            leftover_segments.append(leftover_w)
            leftover_coordinates.append((0, d_end, 0, h_end, w_start, w_end))
    
    return main_segments, main_coordinates, leftover_segments, leftover_coordinates

In [119]:
main_segments, main_coords, leftover_segments, leftover_coords = segment_3d_with_coordinates(
    original_image,
    partition_d = 2,
    partition_h = 2,
    partition_w = 2,
    )


# Access segments and their coordinates
for segment, coords in zip(main_segments, main_coords):
    d_start, d_end, h_start, h_end, w_start, w_end = coords
    print(f"Segment shape: {segment.shape}")
    print(f"Coordinates: ({d_start}:{d_end}, {h_start}:{h_end}, {w_start}:{w_end})")


Segment shape: (195, 582, 512)
Coordinates: (0:195, 0:582, 0:512)
Segment shape: (195, 582, 512)
Coordinates: (0:195, 0:582, 512:1024)
Segment shape: (195, 582, 512)
Coordinates: (0:195, 582:1164, 0:512)
Segment shape: (195, 582, 512)
Coordinates: (0:195, 582:1164, 512:1024)
Segment shape: (195, 582, 512)
Coordinates: (195:390, 0:582, 0:512)
Segment shape: (195, 582, 512)
Coordinates: (195:390, 0:582, 512:1024)
Segment shape: (195, 582, 512)
Coordinates: (195:390, 582:1164, 0:512)
Segment shape: (195, 582, 512)
Coordinates: (195:390, 582:1164, 512:1024)


In [120]:
# Print results
print(f"Original data shape: {original_image.shape}")
print(f"Number of main segments: {len(main_segments)}")
print("\nMain segment coordinates:")
for i, coords in enumerate(main_coords):
    print(f"Segment {i}: {coords}")

print("\nLeftover segment coordinates:")
for i, coords in enumerate(leftover_coords):
    print(f"Leftover {i}: {coords}")

Original data shape: (390, 1164, 1024)
Number of main segments: 8

Main segment coordinates:
Segment 0: (0, 195, 0, 582, 0, 512)
Segment 1: (0, 195, 0, 582, 512, 1024)
Segment 2: (0, 195, 582, 1164, 0, 512)
Segment 3: (0, 195, 582, 1164, 512, 1024)
Segment 4: (195, 390, 0, 582, 0, 512)
Segment 5: (195, 390, 0, 582, 512, 1024)
Segment 6: (195, 390, 582, 1164, 0, 512)
Segment 7: (195, 390, 582, 1164, 512, 1024)

Leftover segment coordinates:


In [123]:
def adjust_segments(main_coords, leftover_coords, image_shape, overlap_percentage = 0.1/2):
    """
    Adjust the segment coordinates with specified expansion or contraction.

    Parameters:
        main_coords (list of tuples): Original main segment coordinates.
        leftover_coords (list of tuples): Original leftover segment coordinates.
        image_shape (tuple): Shape of the original image as (z, y, x).

    Returns:
        adjusted_main_coords (list of tuples): Adjusted main segment coordinates.
        adjusted_leftover_coords (list of tuples): Adjusted leftover segment coordinates.
    """
    z_max, y_max, x_max = image_shape

    z_overlap = int(overlap_percentage * z_max)
    y_overlap = int(overlap_percentage * y_max)
    x_overlap = int(overlap_percentage * x_max)

    # Adjust main segment coordinates
    adjusted_main_coords = []
    for i, (z_start, z_end, y_start, y_end, x_start, x_end) in enumerate(main_coords):
        z_start_adj = max(0, z_start - z_overlap)
        z_end_adj = min(z_max, z_end + z_overlap)
        y_start_adj = max(0, y_start - y_overlap)
        y_end_adj = min(y_max, y_end + y_overlap)
        x_start_adj = max(0, x_start - x_overlap)
        x_end_adj = min(x_max, x_end + x_overlap)

        # If boundary constraints apply, adjust
        if z_end_adj == z_max:
            z_end_adj = z_max
        if y_end_adj == y_max:
            y_end_adj = y_max
        if x_end_adj == x_max:
            x_end_adj = x_max

        adjusted_main_coords.append((z_start_adj, z_end_adj, y_start_adj, y_end_adj, x_start_adj, x_end_adj))

    # Adjust leftover segment coordinates
    adjusted_leftover_coords = []
    for i, (z_start, z_end, y_start, y_end, x_start, x_end) in enumerate(leftover_coords):
        # Assume leftover segments retain original coordinates but apply boundaries
        z_start_adj = z_start
        z_end_adj = z_max if z_end == z_max else z_end
        y_start_adj = y_start
        y_end_adj = y_max if y_end == y_max else y_end
        x_start_adj = x_start
        x_end_adj = x_end

        adjusted_leftover_coords.append((z_start_adj, z_end_adj, y_start_adj, y_end_adj, x_start_adj, x_end_adj))

    return adjusted_main_coords, adjusted_leftover_coords

# # Example usage
# main_coords = [
#     (0, 195, 0, 512, 0, 582),
#     (0, 195, 512, 1024, 0, 582),
#     (195, 390, 0, 512, 0, 582),
#     (195, 390, 512, 1024, 0, 582)
# ]
# leftover_coords = [
#     (0, 390, 1024, 1164, 0, 582),
#     (0, 390, 0, 1024, 582, 1024)
# ]

original_image_shape = original_image.shape

adjusted_main, adjusted_leftover = adjust_segments(main_coords, leftover_coords, original_image_shape)

# Print adjusted results
print("Adjusted Main Segment Coordinates:")
for i, coords in enumerate(adjusted_main):
    print(f"Segment {i}: {coords}")

print("\nAdjusted Leftover Segment Coordinates:")
for i, coords in enumerate(adjusted_leftover):
    print(f"Leftover {i}: {coords}")


Adjusted Main Segment Coordinates:
Segment 0: (0, 214, 0, 640, 0, 563)
Segment 1: (0, 214, 0, 640, 461, 1024)
Segment 2: (0, 214, 524, 1164, 0, 563)
Segment 3: (0, 214, 524, 1164, 461, 1024)
Segment 4: (176, 390, 0, 640, 0, 563)
Segment 5: (176, 390, 0, 640, 461, 1024)
Segment 6: (176, 390, 524, 1164, 0, 563)
Segment 7: (176, 390, 524, 1164, 461, 1024)

Adjusted Leftover Segment Coordinates:


In [124]:
import numpy as np
import tifffile as tiff

# Load the 3D TIFF image
image = original_image

# # List of coordinates
# adjusted_main = [
#     (0, 200, 0, 682, 0, 612),
#     (0, 200, 0, 682, 412, 1024),
#     (0, 200, 482, 1164, 0, 612),
#     (0, 200, 482, 1164, 412, 1024),
#     (190, 390, 0, 682, 0, 612),
#     (190, 390, 0, 682, 412, 1024),
#     (190, 390, 482, 1164, 0, 612),
#     (190, 390, 482, 1164, 412, 1024)
# ]

# Process each crop
cropped_images = []
for i, (z_start, z_end, y_start, y_end, x_start, x_end) in enumerate(adjusted_main):
    cropped = image[z_start:z_end, y_start:y_end, x_start:x_end]
    cropped_images.append(cropped)
    # Save each cropped section if needed
    tiff.imwrite(f'output\cropped_section_{i}.tif', cropped)

print(f"{len(cropped_images)} cropped images saved successfully.")


8 cropped images saved successfully.
